## Artificial Intelligence Assignment 3
## No3. CNN5 Log Likelihood Loss
## 20132651 Sungjae Lee

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [3]:
## 이번 L2 Normalization 의 핵심은 optimizer 에서 어떤 매개변수를 주느냐에 달려있습니다.
## L2 Norm 이라는 것은 결국 가중치에 대해 규제를 거는 것으로, 가중치 감쇠 (weight decay) 입니다.
## 이러한 L2 Norm 은 optimizer 내부에 weight_decay 라는 매개변수를 통해 조정이 가능하며 lambda 값을 지정하여
## 어느정도 세기의 가중치 감쇠를 시행할 것인가를 정하는 것으로 보입니다.

## 여기서는 Quora 의 How could I choose the value of weight decay for neural network regularization?
## 질문에 나타난 대로 0.001, 0.01, 0.1, 1, 10 을 해보라는 조언에 따라 0.001 을 기본 값으로 설정하고 시행하였습니다.

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay = 0.001)

### (3) 화면 출력 확인 및 학습이 되고 있는지 서술

In [4]:
for epoch in range(2):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss : %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

[1,  1000] loss : 2.295
[1,  2000] loss : 2.114
[1,  3000] loss : 1.928
[1,  4000] loss : 1.817
[1,  5000] loss : 1.727
[1,  6000] loss : 1.649
[1,  7000] loss : 1.611
[1,  8000] loss : 1.584
[1,  9000] loss : 1.586
[1, 10000] loss : 1.550
[1, 11000] loss : 1.528
[1, 12000] loss : 1.480
[2,  1000] loss : 1.450
[2,  2000] loss : 1.433
[2,  3000] loss : 1.417
[2,  4000] loss : 1.425
[2,  5000] loss : 1.374
[2,  6000] loss : 1.352
[2,  7000] loss : 1.318
[2,  8000] loss : 1.320
[2,  9000] loss : 1.341
[2, 10000] loss : 1.308
[2, 11000] loss : 1.304
[2, 12000] loss : 1.275
Finished Training


### (6) 화면 출력 확인 및 일반화 성능 서술

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images :%d %%'%(100 * correct / total))

Accuracy of the network on the 10000 test images :53 %


In [6]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

### (7) 화면 출력 확인 및 부류별 분류기의 성능 서술

In [7]:
for i in range(10):
    print('Accuracy of %5s: %2d %%'%(classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane: 55 %
Accuracy of   car: 81 %
Accuracy of  bird: 23 %
Accuracy of   cat: 38 %
Accuracy of  deer: 46 %
Accuracy of   dog: 30 %
Accuracy of  frog: 61 %
Accuracy of horse: 80 %
Accuracy of  ship: 68 %
Accuracy of truck: 53 %
